# Attempt 1
### Straight-forward using the existing sheet renamed to "T0 vs T1 Prices.csv"

In [1]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [2]:
df = pd.read_csv("T0 vs T1 Prices.csv")
df.head()

,join_key,Name,Current Price,Units,Allocated_Budget,t_1_price,Proft/Loss,Diff,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,504084.0___NA__,Kaycee Inds.,63077.40,1.0,63077.4,69368.56,6291.16,6291.16,NaN,NaN,NaN
1,523395.0_3MINDIA,3M India,37300.00,0.0,0.0,41655.06,0.00,4355.06,NaN,NaN,NaN
2,500068.0___NA__,Disa India,15731.30,0.0,0.0,17431.69,0.00,1700.39,NaN,NaN,NaN
3,505412.0_WENDT,Wendt India,15586.75,0.0,0.0,16910.89,0.00,1324.14,NaN,NaN,NaN
4,509152.0_GRPLTD,GRP,15007.65,0.0,0.0,16027.45,0.00,1019.80,NaN,NaN,NaN


In [3]:
df.tail()

,join_key,Name,Current Price,Units,Allocated_Budget,t_1_price,Proft/Loss,Diff,Unnamed: 8,Unnamed: 9,Unnamed: 10
622,512591.0___NA__,Pulsar Intl.,14.04,10.0,140.40,14.00,-0.40,-0.04,NaN,NaN,NaN
623,-1.0_AAKASH,Aakash Explor.,14.16,12.0,169.92,13.30,-10.32,-0.86,NaN,NaN,NaN
624,511185.0___NA__,Rajvi Logitrade,11.29,0.0,0.00,10.63,-123.42,-0.66,NaN,NaN,NaN
625,NaN,NaN,NaN,NaN,954759.72,NaN,62494.46,NaN,NaN,NaN,6.545569392
626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN


In [4]:
df = df.loc[: 624, ["Name", "Current Price", "t_1_price"]]
# df.tail()

In [5]:
df.columns = ["Name", "t0", "t1"]

In [6]:
df["p10"] = df["t1"] - df["t0"]
df.tail()

,Name,t0,t1,p10
620,RattanIndia Pow.,17.80,18.85,1.05
621,Shahi Shipping,17.12,15.62,-1.50
622,Pulsar Intl.,14.04,14.00,-0.04
623,Aakash Explor.,14.16,13.30,-0.86
624,Rajvi Logitrade,11.29,10.63,-0.66


In [7]:
t0 = df['t0'].values
t1 = df['t1'].values
p = df["p10"].values

In [16]:
n_companies = len(df)

# Define the problem
prob = LpProblem("Maximize_Profit", LpMaximize)
# Define decision variables
shares = [LpVariable(f"shares_{i}", lowBound = 0, cat = 'Integer') for i in range(n_companies)]
# Objective function
prob += lpSum([p[i] * shares[i] for i in range(n_companies)]), "Total_Profit"
# Constraint: Total investment should be less than or equal to 10 lakhs
prob += lpSum([t0[i] * shares[i] for i in range(n_companies)]) <= 1000000, "Total_Investment"
# Solve the problem
prob.solve()

# Print the results
print("Optimal number of shares to buy for each company:")
total_cost = 0
total_profit = 0
shares_allocation = []
for i in range(n_companies):
    shares_allocation.append(int(shares[i].varValue))
    if shares[i].varValue > 0:
        print(f"Company {df['Name'][i]}: {shares[i].varValue} shares")
        total_cost += t0[i] * shares[i].varValue
        total_profit += p[i] * shares[i].varValue

# total_profit = sum([p[i] * shares[i].varValue for i in range(n_companies)])
print(f"Total Cost: {total_cost:.2f}")
print(f"Total Profit: {total_profit:.2f}")

df["allocation"] = shares_allocation
df.head()

Optimal number of shares to buy for each company:
Company Bondada Engineer: 377.0 shares
Company Premier Road: 44.0 shares
Total Cost: 999986.20
Total Profit: 222106.71


,Name,t0,t1,p10,allocation
0,Kaycee Inds.,63077.40,69368.56,6291.16,0
1,3M India,37300.00,41655.06,4355.06,0
2,Disa India,15731.30,17431.69,1700.39,0
3,Wendt India,15586.75,16910.89,1324.14,0
4,GRP,15007.65,16027.45,1019.80,0


In [18]:
df.to_csv("ATTEMPT 1.csv", index = False)

# Attempt 2
### Using regression to predict T2 prices and linear programming to maximize profits p20

## Data Processing

In [22]:
# Balance Sheet
df = pd.read_csv("Balance_Sheet_final.csv")
df = df[["Name", "join_key", "Market Capitalization", "Current Price"]]
balance_sheet_df = df[df["Name"] != "Divine Power"]
print(len(balance_sheet_df))
balance_sheet_df.head()

4667


,Name,join_key,Market Capitalization,Current Price
0,20 Microns,533022.0_20MICRONS,795.54,225.45
1,21st Cent. Mgmt.,526921.0_21STCENMGM,73.96,70.44
2,360 ONE,542772.0_360ONE,36254.08,999.10
3,3B Blackbio,532067.0___NA__,1024.98,1194.20
4,3C IT Solutions,544190.0___NA__,26.55,44.10


In [23]:
# T1 Prices
df = pd.read_csv("t1_prices.csv")
t1_df = df[df["join_key"] != "-1.0___NA__"]
print(len(t1_df))
t1_df.head()

4667


,join_key,t_1_price
0,533022.0_20MICRONS,229.42
1,526921.0_21STCENMGM,72.33
2,542772.0_360ONE,1025.83
3,532067.0___NA__,1329.94
4,544190.0___NA__,45.14


In [24]:
# Ratios 1
df = pd.read_csv("ratios_1_final.csv")
df = df[["join_key", "Piotroski score", "G Factor"]]
ratios1_df = df[df["join_key"] != "-1.0___NA__"]
print(len(ratios1_df))
ratios1_df.head()

4667


,join_key,Piotroski score,G Factor
0,533022.0_20MICRONS,8.0,6.0
1,526921.0_21STCENMGM,6.0,5.0
2,542772.0_360ONE,3.0,6.0
3,532067.0___NA__,5.0,3.0
4,544190.0___NA__,3.0,2.0


In [25]:
# Ratios 2
df = pd.read_csv("ratios_2_final.csv")
df = df[["join_key", "Average return on capital employed 3Years", "Average return on capital employed 5Years", "Average return on capital employed 7Years", "Average return on capital employed 10Years", "Average debtor days 3years", "Average Working Capital Days 3years"]]
ratios2_df = df[df["join_key"] != "-1.0___NA__"]
print(len(ratios2_df))
ratios2_df.head()

4667


,join_key,Average return on capital employed 3Years,Average return on capital employed 5Years,Average return on capital employed 7Years,Average return on capital employed 10Years,Average debtor days 3years,Average Working Capital Days 3years
0,533022.0_20MICRONS,20.14,18.86,19.18,17.75,56.42,84.01
1,526921.0_21STCENMGM,-1.17,-6.00,-0.97,4.67,89.26,51.05
2,542772.0_360ONE,13.73,11.58,11.46,11.46,47.33,-109.10
3,532067.0___NA__,22.23,51.94,44.46,33.72,171.51,210.92
4,544190.0___NA__,27.48,27.48,27.48,27.48,66.12,64.30


In [26]:
# Price
df = pd.read_csv("price_final.csv")
df = df[["join_key", "Volume 1month average", "Volume 1week average", "Volume", "Volume 1year average"]]
price_df = df[df["join_key"] != "-1.0___NA__"]
print(len(price_df))
price_df.head()

4667


,join_key,Volume 1month average,Volume 1week average,Volume,Volume 1year average
0,533022.0_20MICRONS,729846,816052,312746,203559
1,526921.0_21STCENMGM,34120,27069,11745,13416
2,542772.0_360ONE,1193124,2871837,5329590,705325
3,532067.0___NA__,21265,5355,7221,13985
4,544190.0___NA__,159500,81000,74000,159500


In [27]:
# Other Metrics
df = pd.read_csv("other_metrics_final.csv")
df = df[["join_key", "ROCE3yr avg"]]
other_metrics_df = df[df["join_key"] != "-1.0___NA__"]
print(len(other_metrics_df))
other_metrics_df.head()

4667


,join_key,ROCE3yr avg
0,533022.0_20MICRONS,20.14
1,526921.0_21STCENMGM,-1.17
2,542772.0_360ONE,13.73
3,532067.0___NA__,22.23
4,544190.0___NA__,27.48


In [99]:
# Final df for attempt 2
df = balance_sheet_df
other_dfs = [t1_df, ratios1_df, ratios2_df, price_df, other_metrics_df]
for df2 in other_dfs:
    df = pd.merge(df, df2, on = "join_key", how = "inner")
print(len(df))
df.head()

4667


,Name,join_key,Market Capitalization,Current Price,t_1_price,Piotroski score,G Factor,Average return on capital employed 3Years,Average return on capital employed 5Years,Average return on capital employed 7Years,Average return on capital employed 10Years,Average debtor days 3years,Average Working Capital Days 3years,Volume 1month average,Volume 1week average,Volume,Volume 1year average,ROCE3yr avg
0,20 Microns,533022.0_20MICRONS,795.54,225.45,229.42,8.0,6.0,20.14,18.86,19.18,17.75,56.42,84.01,729846,816052,312746,203559,20.14
1,21st Cent. Mgmt.,526921.0_21STCENMGM,73.96,70.44,72.33,6.0,5.0,-1.17,-6.00,-0.97,4.67,89.26,51.05,34120,27069,11745,13416,-1.17
2,360 ONE,542772.0_360ONE,36254.08,999.10,1025.83,3.0,6.0,13.73,11.58,11.46,11.46,47.33,-109.10,1193124,2871837,5329590,705325,13.73
3,3B Blackbio,532067.0___NA__,1024.98,1194.20,1329.94,5.0,3.0,22.23,51.94,44.46,33.72,171.51,210.92,21265,5355,7221,13985,22.23
4,3C IT Solutions,544190.0___NA__,26.55,44.10,45.14,3.0,2.0,27.48,27.48,27.48,27.48,66.12,64.30,159500,81000,74000,159500,27.48


In [29]:
df.to_csv("attempt 2 df.csv", index = False)

## Train Test Split

In [35]:
from sklearn.model_selection import train_test_split

In [100]:
y = df["t_1_price"]
X = df.drop(["Name", "join_key", "t_1_price"], axis = 1)
print(X.shape, y.shape)

(4667, 15) (4667,)


In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(3266, 15) (1401, 15)
(3266,) (1401,)


## Regression

In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [50]:
from sklearn.metrics import mean_squared_error, r2_score

In [31]:
# y = df["t_1_price"]
# X = df.drop(["Name", "join_key", "t_1_price"], axis = 1)
# print(X.shape, y.shape)

(4667, 15) (4667,)


In [88]:
models = [LinearRegression(), Ridge(alpha = 0.1), DecisionTreeRegressor(random_state = 0), RandomForestRegressor(random_state = 0)]
best_model = None
best_r2 = None
best_i = 0
for i, model in enumerate(models, 1):
    model.fit(X_train, y_train)
    yhat = model.predict(X_test)
    mse = mean_squared_error(y_test, yhat)
    r2 = r2_score(y_test, yhat)
    print(f"MODEL - {i}:")
    print(f"MSE = {mse}")
    print(f"R2 Score = {r2}")
    print()

    if not best_model or r2 > best_r2:
        best_model = model
        best_r2 = r2
        best_i = i
print(f"Best model = Model - {best_i}")

MODEL - 1:
MSE = 30571.49537409865
R2 Score = 0.9971039469453501

MODEL - 2:
MSE = 30571.49529671456
R2 Score = 0.9971039469526808

MODEL - 3:
MSE = 2916087.8064882937
R2 Score = 0.7237575428919731



c:\Users\iitsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=2.40957e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


MODEL - 4:
MSE = 211797.345099339
R2 Score = 0.9799363315161432

Best model = Model - 2


## Inference (Getting T2 Prices)

In [102]:
# Set T1 prices as the current price for predicting the T2 prices
t0 = X["Current Price"].values
t1 = y.values
X["Current Price"] = y
t2 = best_model.predict(X)

In [54]:
# type(t2)

numpy.ndarray

In [103]:
# Create the dataframe of the prices T0 to T2
t0t1t2_df = pd.DataFrame()
t0t1t2_df[["Name", "join_key"]] = df[["Name", "join_key"]]
t0t1t2_df["t0"] = t0
t0t1t2_df["t1"] = t1
t0t1t2_df["t2"] = t2
print(t0t1t2_df.shape)
t0t1t2_df.head()

(4667, 5)


,Name,join_key,t0,t1,t2
0,20 Microns,533022.0_20MICRONS,225.45,229.42,260.447690
1,21st Cent. Mgmt.,526921.0_21STCENMGM,70.44,72.33,86.484221
2,360 ONE,542772.0_360ONE,999.10,1025.83,1068.317392
3,3B Blackbio,532067.0___NA__,1194.20,1329.94,1371.522189
4,3C IT Solutions,544190.0___NA__,44.10,45.14,34.605736


In [84]:
# t0t1t2_df[t0t1t2_df["t2"] != t0t1t2_df["t1"]]

,Name,join_key,t0,t1,t2
0,20 Microns,533022.0_20MICRONS,225.45,229.42,229.42
1,21st Cent. Mgmt.,526921.0_21STCENMGM,70.44,72.33,72.33
2,360 ONE,542772.0_360ONE,999.10,1025.83,1025.83
3,3B Blackbio,532067.0___NA__,1194.20,1329.94,1329.94
4,3C IT Solutions,544190.0___NA__,44.10,45.14,45.14
...,...,...,...,...,...
4662,Zota Health Care,-1.0_ZOTA,573.40,550.89,550.89
4663,Zuari Agro Chem.,534742.0_ZUARI,226.70,197.72,197.72
4664,Zuari Industries,500780.0_ZUARIIND,403.90,434.14,434.14
4665,Zydus Lifesci.,532321.0_ZYDUSLIFE,1069.30,1144.44,1144.44


In [104]:
t0t1t2_df.to_csv("T0 vs T1 vs T2 Prices.csv", index = False)

## Maximizing Profits

In [105]:
df = t0t1t2_df
df["p20"] = df["t2"] - df["t0"]
df["p21"] = df["t2"] - df["t1"]
df["p10"] = df["t1"] - df["t0"]
df.head()

,Name,join_key,t0,t1,t2,p20,p21,p10
0,20 Microns,533022.0_20MICRONS,225.45,229.42,260.447690,34.997690,31.027690,3.97
1,21st Cent. Mgmt.,526921.0_21STCENMGM,70.44,72.33,86.484221,16.044221,14.154221,1.89
2,360 ONE,542772.0_360ONE,999.10,1025.83,1068.317392,69.217392,42.487392,26.73
3,3B Blackbio,532067.0___NA__,1194.20,1329.94,1371.522189,177.322189,41.582189,135.74
4,3C IT Solutions,544190.0___NA__,44.10,45.14,34.605736,-9.494264,-10.534264,1.04


In [106]:
# Get the best allocation based on an upper limit on the number of shares allocated per company/stock
n_companies = len(df)
upper_bounds = [100, 200, 500, 1000, 2000, 3000, 4000, 5000, 8000, 10000]
allocations = dict()
best_profit_10 = [None, None]
best_profit_21 = [None, None]
best_profit_20 = [None, None]
# best_profit_total = [None, None]

for ub in upper_bounds:
    # Define the problem
    prob = LpProblem("Maximize_Profit", LpMaximize)
    # Define decision variables
    shares = [LpVariable(f"shares_{i}", lowBound = 0, upBound = ub, cat = 'Integer') for i in range(n_companies)]
    # Objective function
    prob += lpSum([df["p20"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T20"
    # prob += lpSum([df["p21"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T21"
    # prob += lpSum([df["p10"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T10"
    # Constraint: Total investment should be less than or equal to 10 lakhs
    # prob += lpSum([df["t1"][i] * shares[i] for i in range(n_companies)]) <= 1000000, "Total_Investment_T1"
    prob += lpSum([df["t0"][i] * shares[i] for i in range(n_companies)]) <= 1000000, "Total_Investment_T0_Max"
    # prob += lpSum([df["t0"][i] * shares[i] for i in range(n_companies)]) >= 900000, "Total_Investment_T0_Min"
    # Solve the problem
    prob.solve()

    # Print the results
    print("Optimal number of shares to buy for each company:")
    total_cost_10 = 0
    total_profit_10 = 0
    total_cost_21 = 0
    total_profit_21 = 0
    total_profit_20 = 0
    shares_allocation = []
    for i in range(n_companies):
        shares_allocation.append(int(shares[i].varValue))
        if shares[i].varValue > 0:
            # print(f"Company {df['Name'][i]}: {shares[i].varValue} shares")
            total_cost_10 += df["t0"][i] * shares[i].varValue
            total_profit_10 += df["p10"][i] * shares[i].varValue
            total_cost_21 += df["t1"][i] * shares[i].varValue
            total_profit_21 += df["p21"][i] * shares[i].varValue
            total_profit_20 += df["p20"][i] * shares[i].varValue

    # total_profit = sum([p[i] * shares[i].varValue for i in range(n_companies)])
    # total_profit = total_profit_10 + total_profit_21
    allocations[ub] = shares_allocation
    print("Upper Bound on each allocation (number of shares): ", ub)
    print(f"Total Cost at T0: {total_cost_10:.2f}")
    print(f"Total Profit from T0 to T1: {total_profit_10:.2f}")
    # print()
    print(f"Total Cost at T1: {total_cost_21:.2f}")
    print(f"Total Profit from T1 to T2: {total_profit_21:.2f}")
    print(f"Total Profit from T0 to T2 (Overall): {total_profit_20:.2f}")
    print()

    if best_profit_10[0] is None or total_profit_10 > best_profit_10[1]:
        best_profit_10[0] = ub
        best_profit_10[1] = total_profit_10
    if best_profit_21[0] is None or total_profit_21 > best_profit_21[1]:
        best_profit_21[0] = ub
        best_profit_21[1] = total_profit_21
    if best_profit_20[0] is None or total_profit_20 > best_profit_20[1]:
        best_profit_20[0] = ub
        best_profit_20[1] = total_profit_20

print(f"Best Profit (T1 - T0): {best_profit_10}")
print(f"Best Profit (T2 - T1): {best_profit_21}")
print(f"Best Overall Profit (T2 - T0): {best_profit_20}")

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  100
Total Cost at T0: 999999.83
Total Profit from T0 to T1: -824.46
Total Cost at T1: 999175.37
Total Profit from T1 to T2: 677266.96
Total Profit from T0 to T2 (Overall): 676442.50

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  200
Total Cost at T0: 999999.98
Total Profit from T0 to T1: -9269.65
Total Cost at T1: 990730.33
Total Profit from T1 to T2: 968725.41
Total Profit from T0 to T2 (Overall): 959455.76

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  500
Total Cost at T0: 999999.95
Total Profit from T0 to T1: -19503.31
Total Cost at T1: 980496.64
Total Profit from T1 to T2: 1499741.99
Total Profit from T0 to T2 (Overall): 1480238.68

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  1000
Total Cost at T0: 1000000.00
Tota

In [107]:
df[(df["p20"] >= 0) & (df["p10"] <= 0)]

,Name,join_key,t0,t1,t2,p20,p21,p10
7,3P Land Hold.,516092.0_3PLAND,38.18,36.98,46.057839,7.877839,9.077839,-1.20
8,52 Weeks Enter.,531925.0___NA__,1.69,1.55,2.434830,0.744830,0.884830,-0.14
14,A B Infrabuild,-1.0_ABINFRA,55.00,54.76,60.678745,5.678745,5.918745,-0.24
16,A F Enterprises,538351.0___NA__,11.30,11.03,11.664065,0.364065,0.634065,-0.27
21,A.K.Capital Serv,530499.0___NA__,1050.00,1017.94,1060.896717,10.896717,42.956717,-32.06
...,...,...,...,...,...,...,...,...
4635,Yuranus Infrast.,536846.0___NA__,127.00,123.88,140.183795,13.183795,16.303795,-3.12
4636,Yuvraaj Hygiene,531663.0___NA__,1.78,1.77,7.189048,5.409048,5.419048,-0.01
4643,Zee Learn,533287.0_ZEELEARN,7.74,7.25,32.069971,24.329971,24.819971,-0.49
4648,Zenith Fibres,514266.0___NA__,85.00,83.34,92.967500,7.967500,9.627500,-1.66


In [108]:
df["allocation"] = allocations[10000]
df.head()

,Name,join_key,t0,t1,t2,p20,p21,p10,allocation
0,20 Microns,533022.0_20MICRONS,225.45,229.42,260.447690,34.997690,31.027690,3.97,0
1,21st Cent. Mgmt.,526921.0_21STCENMGM,70.44,72.33,86.484221,16.044221,14.154221,1.89,0
2,360 ONE,542772.0_360ONE,999.10,1025.83,1068.317392,69.217392,42.487392,26.73,0
3,3B Blackbio,532067.0___NA__,1194.20,1329.94,1371.522189,177.322189,41.582189,135.74,0
4,3C IT Solutions,544190.0___NA__,44.10,45.14,34.605736,-9.494264,-10.534264,1.04,0


In [110]:
sum(df["allocation"] > 0)

46

In [111]:
df.to_csv("ATTEMPT 2.csv", index = False)

# Attempt 3
### Maximizing profits p20 (the actual desired profits) as well as p10

In [112]:
# Get the best allocation based on an upper limit on the number of shares allocated per company/stock
n_companies = len(df)
upper_bounds = [100, 200, 500, 1000, 2000, 3000, 4000, 5000, 8000, 10000]
allocations = dict()
best_profit_10 = [None, None]
best_profit_21 = [None, None]
best_profit_20 = [None, None]
# best_profit_total = [None, None]

for ub in upper_bounds:
    # Define the problem
    prob = LpProblem("Maximize_Profit", LpMaximize)
    # Define decision variables
    shares = [LpVariable(f"shares_{i}", lowBound = 0, upBound = ub, cat = 'Integer') for i in range(n_companies)]
    # Objective function
    prob += lpSum([df["p20"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T20"
    # prob += lpSum([df["p21"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T21"
    prob += lpSum([df["p10"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T10"
    # Constraint: Total investment should be less than or equal to 10 lakhs
    # prob += lpSum([df["t1"][i] * shares[i] for i in range(n_companies)]) <= 1000000, "Total_Investment_T1"
    prob += lpSum([df["t0"][i] * shares[i] for i in range(n_companies)]) <= 1000000, "Total_Investment_T0_Max"
    # prob += lpSum([df["t0"][i] * shares[i] for i in range(n_companies)]) >= 900000, "Total_Investment_T0_Min"
    # Solve the problem
    prob.solve()

    # Print the results
    print("Optimal number of shares to buy for each company:")
    total_cost_10 = 0
    total_profit_10 = 0
    total_cost_21 = 0
    total_profit_21 = 0
    total_profit_20 = 0
    shares_allocation = []
    for i in range(n_companies):
        shares_allocation.append(int(shares[i].varValue))
        if shares[i].varValue > 0:
            # print(f"Company {df['Name'][i]}: {shares[i].varValue} shares")
            total_cost_10 += df["t0"][i] * shares[i].varValue
            total_profit_10 += df["p10"][i] * shares[i].varValue
            total_cost_21 += df["t1"][i] * shares[i].varValue
            total_profit_21 += df["p21"][i] * shares[i].varValue
            total_profit_20 += df["p20"][i] * shares[i].varValue

    # total_profit = sum([p[i] * shares[i].varValue for i in range(n_companies)])
    # total_profit = total_profit_10 + total_profit_21
    allocations[ub] = shares_allocation
    print("Upper Bound on each allocation (number of shares): ", ub)
    print(f"Total Cost at T0: {total_cost_10:.2f}")
    print(f"Total Profit from T0 to T1: {total_profit_10:.2f}")
    # print()
    print(f"Total Cost at T1: {total_cost_21:.2f}")
    print(f"Total Profit from T1 to T2: {total_profit_21:.2f}")
    print(f"Total Profit from T0 to T2 (Overall): {total_profit_20:.2f}")
    print()

    if best_profit_10[0] is None or total_profit_10 > best_profit_10[1]:
        best_profit_10[0] = ub
        best_profit_10[1] = total_profit_10
    if best_profit_21[0] is None or total_profit_21 > best_profit_21[1]:
        best_profit_21[0] = ub
        best_profit_21[1] = total_profit_21
    if best_profit_20[0] is None or total_profit_20 > best_profit_20[1]:
        best_profit_20[0] = ub
        best_profit_20[1] = total_profit_20

print(f"Best Profit (T1 - T0): {best_profit_10}")
print(f"Best Profit (T2 - T1): {best_profit_21}")
print(f"Best Overall Profit (T2 - T0): {best_profit_20}")

c:\Users\iitsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  100
Total Cost at T0: 999999.98
Total Profit from T0 to T1: 254856.08
Total Cost at T1: 1254856.06
Total Profit from T1 to T2: 43951.82
Total Profit from T0 to T2 (Overall): 298807.90

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  200
Total Cost at T0: 999999.94
Total Profit from T0 to T1: 270577.30
Total Cost at T1: 1270577.24
Total Profit from T1 to T2: 43454.71
Total Profit from T0 to T2 (Overall): 314032.01

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  500
Total Cost at T0: 999999.94
Total Profit from T0 to T1: 270619.62
Total Cost at T1: 1270619.56
Total Profit from T1 to T2: 48079.37
Total Profit from T0 to T2 (Overall): 318698.99

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  1000
Total Cost at T0: 999999.97
Tota

In [113]:
df["allocation"] = allocations[10000]
df.head()

,Name,join_key,t0,t1,t2,p20,p21,p10,allocation
0,20 Microns,533022.0_20MICRONS,225.45,229.42,260.447690,34.997690,31.027690,3.97,0
1,21st Cent. Mgmt.,526921.0_21STCENMGM,70.44,72.33,86.484221,16.044221,14.154221,1.89,0
2,360 ONE,542772.0_360ONE,999.10,1025.83,1068.317392,69.217392,42.487392,26.73,0
3,3B Blackbio,532067.0___NA__,1194.20,1329.94,1371.522189,177.322189,41.582189,135.74,0
4,3C IT Solutions,544190.0___NA__,44.10,45.14,34.605736,-9.494264,-10.534264,1.04,0


In [114]:
sum(df["allocation"] > 0)

5

In [115]:
df.to_csv("ATTEMPT 3.csv", index = False)

# Attempt 4
### Choose smaller upper bound on stock allocations for lower risk

In [116]:
df["allocation"] = allocations[500]
df.head()

,Name,join_key,t0,t1,t2,p20,p21,p10,allocation
0,20 Microns,533022.0_20MICRONS,225.45,229.42,260.447690,34.997690,31.027690,3.97,0
1,21st Cent. Mgmt.,526921.0_21STCENMGM,70.44,72.33,86.484221,16.044221,14.154221,1.89,0
2,360 ONE,542772.0_360ONE,999.10,1025.83,1068.317392,69.217392,42.487392,26.73,0
3,3B Blackbio,532067.0___NA__,1194.20,1329.94,1371.522189,177.322189,41.582189,135.74,0
4,3C IT Solutions,544190.0___NA__,44.10,45.14,34.605736,-9.494264,-10.534264,1.04,0


In [117]:
sum(df["allocation"] > 0)

4

In [118]:
df.to_csv("ATTEMPT 4.csv", index = False)

# Attempt 5
### Employing the same algorithm from attempt 2 on the safe companies

## Data Processing

In [119]:
df = pd.read_csv("attempt 2 df.csv")
df.head()

,Name,join_key,Market Capitalization,Current Price,t_1_price,Piotroski score,G Factor,Average return on capital employed 3Years,Average return on capital employed 5Years,Average return on capital employed 7Years,Average return on capital employed 10Years,Average debtor days 3years,Average Working Capital Days 3years,Volume 1month average,Volume 1week average,Volume,Volume 1year average,ROCE3yr avg
0,20 Microns,533022.0_20MICRONS,795.54,225.45,229.42,8.0,6.0,20.14,18.86,19.18,17.75,56.42,84.01,729846,816052,312746,203559,20.14
1,21st Cent. Mgmt.,526921.0_21STCENMGM,73.96,70.44,72.33,6.0,5.0,-1.17,-6.00,-0.97,4.67,89.26,51.05,34120,27069,11745,13416,-1.17
2,360 ONE,542772.0_360ONE,36254.08,999.10,1025.83,3.0,6.0,13.73,11.58,11.46,11.46,47.33,-109.10,1193124,2871837,5329590,705325,13.73
3,3B Blackbio,532067.0___NA__,1024.98,1194.20,1329.94,5.0,3.0,22.23,51.94,44.46,33.72,171.51,210.92,21265,5355,7221,13985,22.23
4,3C IT Solutions,544190.0___NA__,26.55,44.10,45.14,3.0,2.0,27.48,27.48,27.48,27.48,66.12,64.30,159500,81000,74000,159500,27.48


In [124]:
safe_companies = pd.read_csv("safe_companies.csv")
safe_companies = safe_companies[["Name", "Price to Sales_x", "EVEBITDA_x", "Working Capital to Sales ratio", "QoQ Profits", "QoQ Sales", "Net worth", "Market Cap to Sales", "Enterprise Value to EBIT", "Debt To Profit", "Total Capital Employed", "Leverage", "Dividend Payout", "Cash by market cap", "52w Index", "Down from 52w high", "Up from 52w low", "From 52w high", "Dividend Payout Ratio", "Market Capt to Cash Flow", "Altman Z Score_x"]]
safe_companies.head()

,Name,Price to Sales_x,EVEBITDA_x,Working Capital to Sales ratio,QoQ Profits,QoQ Sales,Net worth,Market Cap to Sales,Enterprise Value to EBIT,Debt To Profit,...,Leverage,Dividend Payout,Cash by market cap,52w Index,Down from 52w high,Up from 52w low,From 52w high,Dividend Payout Ratio,Market Capt to Cash Flow,Altman Z Score_x
0,20 Microns,1.02,7.80,31.86,18.78,21.41,352.95,1.02,8.97,2.16,...,1.57,7.87,0.06,93.07,4.27,149.89,0.96,7.87,14.58,5.36
1,3M India,10.71,52.94,34.92,29.44,9.66,2370.20,10.71,56.74,0.03,...,1.33,143.96,0.02,80.55,6.46,40.08,0.94,143.96,72.06,23.11
2,A B B,16.24,84.24,40.96,35.62,11.71,5944.60,16.24,89.56,0.04,...,1.71,40.60,0.02,87.27,7.40,121.41,0.93,40.60,133.57,29.11
3,Aakash Explor.,1.55,9.07,19.38,756.10,41.70,58.53,1.55,15.81,2.92,...,1.51,0.00,0.01,87.22,8.05,148.42,0.92,0.00,14.04,4.89
4,Aaron Industries,4.34,26.93,27.44,94.17,30.51,35.84,4.34,30.81,4.99,...,1.58,16.54,0.01,40.87,19.79,20.56,0.80,16.54,48.96,8.11


In [125]:
df = pd.merge(df, safe_companies, on = "Name", how = "inner")
print(len(df))
df.head()

366


,Name,join_key,Market Capitalization,Current Price,t_1_price,Piotroski score,G Factor,Average return on capital employed 3Years,Average return on capital employed 5Years,Average return on capital employed 7Years,...,Leverage,Dividend Payout,Cash by market cap,52w Index,Down from 52w high,Up from 52w low,From 52w high,Dividend Payout Ratio,Market Capt to Cash Flow,Altman Z Score_x
0,20 Microns,533022.0_20MICRONS,795.54,225.45,229.42,8.0,6.0,20.14,18.86,19.18,...,1.57,7.87,0.06,93.07,4.27,149.89,0.96,7.87,14.58,5.36
1,3M India,523395.0_3MINDIA,42048.39,37310.00,41655.06,9.0,4.0,24.53,19.83,23.28,...,1.33,143.96,0.02,80.55,6.46,40.08,0.94,143.96,72.06,23.11
2,A B B,500002.0_ABB,180784.34,8531.25,9620.43,9.0,5.0,23.27,18.23,17.55,...,1.71,40.60,0.02,87.27,7.40,121.41,0.93,40.60,133.57,29.11
3,Aakash Explor.,-1.0_AAKASH,143.37,14.16,13.30,9.0,5.0,13.31,13.61,13.32,...,1.51,0.00,0.01,87.22,8.05,148.42,0.92,0.00,14.04,4.89
4,Aaron Industries,-1.0_AARON,275.65,263.20,258.78,3.0,5.0,24.11,21.16,24.69,...,1.58,16.54,0.01,40.87,19.79,20.56,0.80,16.54,48.96,8.11


In [126]:
df.to_csv("attempt 5 df.csv", index = False)

## Train Test Split

In [127]:
y = df["t_1_price"]
X = df.drop(["Name", "join_key", "t_1_price"], axis = 1)
print(X.shape, y.shape)

(366, 35) (366,)


In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(256, 35) (110, 35)
(256,) (110,)


## Regression

In [129]:
models = [LinearRegression(), Ridge(alpha = 0.1), DecisionTreeRegressor(random_state = 0), RandomForestRegressor(random_state = 0)]
best_model = None
best_r2 = None
best_i = 0
for i, model in enumerate(models, 1):
    model.fit(X_train, y_train)
    yhat = model.predict(X_test)
    mse = mean_squared_error(y_test, yhat)
    r2 = r2_score(y_test, yhat)
    print(f"MODEL - {i}:")
    print(f"MSE = {mse}")
    print(f"R2 Score = {r2}")
    print()

    if not best_model or r2 > best_r2:
        best_model = model
        best_r2 = r2
        best_i = i
print(f"Best model = Model - {best_i}")

MODEL - 1:
MSE = 27737.701590986337
R2 Score = 0.9969966247208891

MODEL - 2:
MSE = 27577.17402525146
R2 Score = 0.9970140062808198

MODEL - 3:
MSE = 16623517.821171816
R2 Score = -0.7999567236021574



c:\Users\iitsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=2.07599e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


MODEL - 4:
MSE = 4592255.497287253
R2 Score = 0.5027610131765428

Best model = Model - 2


## Inference

In [130]:
# Set T1 prices as the current price for predicting the T2 prices
t0 = X["Current Price"].values
t1 = y.values
X["Current Price"] = y
t2 = best_model.predict(X)

In [131]:
# Create the dataframe of the prices T0 to T2
t0t1t2_df = pd.DataFrame()
t0t1t2_df[["Name", "join_key"]] = df[["Name", "join_key"]]
t0t1t2_df["t0"] = t0
t0t1t2_df["t1"] = t1
t0t1t2_df["t2"] = t2
print(t0t1t2_df.shape)
t0t1t2_df.head()

(366, 5)


,Name,join_key,t0,t1,t2
0,20 Microns,533022.0_20MICRONS,225.45,229.42,227.561919
1,3M India,523395.0_3MINDIA,37310.00,41655.06,45885.999153
2,A B B,500002.0_ABB,8531.25,9620.43,10533.824027
3,Aakash Explor.,-1.0_AAKASH,14.16,13.30,-58.956361
4,Aaron Industries,-1.0_AARON,263.20,258.78,295.829774


In [132]:
t0t1t2_df.to_csv("Safe Companies T0 vs T1 vs T2 Prices.csv", index = False)

## Linear Programming

In [133]:
df = t0t1t2_df
df["p20"] = df["t2"] - df["t0"]
df["p21"] = df["t2"] - df["t1"]
df["p10"] = df["t1"] - df["t0"]
df.head()

,Name,join_key,t0,t1,t2,p20,p21,p10
0,20 Microns,533022.0_20MICRONS,225.45,229.42,227.561919,2.111919,-1.858081,3.97
1,3M India,523395.0_3MINDIA,37310.00,41655.06,45885.999153,8575.999153,4230.939153,4345.06
2,A B B,500002.0_ABB,8531.25,9620.43,10533.824027,2002.574027,913.394027,1089.18
3,Aakash Explor.,-1.0_AAKASH,14.16,13.30,-58.956361,-73.116361,-72.256361,-0.86
4,Aaron Industries,-1.0_AARON,263.20,258.78,295.829774,32.629774,37.049774,-4.42


In [140]:
# Get the best allocation based on an upper limit on the number of shares allocated per company/stock
n_companies = len(df)
upper_bounds = [100, 200, 500, 1000, 2000, 3000, 4000, 5000, 8000, 10000]
allocations = dict()
best_profit_10 = [None, None]
best_profit_21 = [None, None]
best_profit_20 = [None, None]
# best_profit_total = [None, None]

for ub in upper_bounds:
    # Define the problem
    prob = LpProblem("Maximize_Profit", LpMaximize)
    # Define decision variables
    shares = [LpVariable(f"shares_{i}", lowBound = 0, upBound = ub, cat = 'Integer') for i in range(n_companies)]
    # Objective function
    prob += lpSum([df["p20"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T20"
    # prob += lpSum([df["p21"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T21"
    # prob += lpSum([df["p10"][i] * shares[i] for i in range(n_companies)]), "Total_Profit_T10"
    # Constraint: Total investment should be less than or equal to 10 lakhs
    # prob += lpSum([df["t1"][i] * shares[i] for i in range(n_companies)]) <= 1000000, "Total_Investment_T1"
    prob += lpSum([df["t0"][i] * shares[i] for i in range(n_companies)]) <= 1000000, "Total_Investment_T0_Max"
    # prob += lpSum([df["t0"][i] * shares[i] for i in range(n_companies)]) >= 900000, "Total_Investment_T0_Min"
    # Solve the problem
    prob.solve()

    # Print the results
    print("Optimal number of shares to buy for each company:")
    total_cost_10 = 0
    total_profit_10 = 0
    total_cost_21 = 0
    total_profit_21 = 0
    total_profit_20 = 0
    shares_allocation = []
    for i in range(n_companies):
        shares_allocation.append(int(shares[i].varValue))
        if shares[i].varValue > 0:
            # print(f"Company {df['Name'][i]}: {shares[i].varValue} shares")
            total_cost_10 += df["t0"][i] * shares[i].varValue
            total_profit_10 += df["p10"][i] * shares[i].varValue
            total_cost_21 += df["t1"][i] * shares[i].varValue
            total_profit_21 += df["p21"][i] * shares[i].varValue
            total_profit_20 += df["p20"][i] * shares[i].varValue

    # total_profit = sum([p[i] * shares[i].varValue for i in range(n_companies)])
    # total_profit = total_profit_10 + total_profit_21
    allocations[ub] = shares_allocation
    print("Upper Bound on each allocation (number of shares): ", ub)
    print(f"Total Cost at T0: {total_cost_10:.2f}")
    print(f"Total Profit from T0 to T1: {total_profit_10:.2f}")
    # print()
    print(f"Total Cost at T1: {total_cost_21:.2f}")
    print(f"Total Profit from T1 to T2: {total_profit_21:.2f}")
    print(f"Total Profit from T0 to T2 (Overall): {total_profit_20:.2f}")
    print()

    if best_profit_10[0] is None or total_profit_10 > best_profit_10[1]:
        best_profit_10[0] = ub
        best_profit_10[1] = total_profit_10
    if best_profit_21[0] is None or total_profit_21 > best_profit_21[1]:
        best_profit_21[0] = ub
        best_profit_21[1] = total_profit_21
    if best_profit_20[0] is None or total_profit_20 > best_profit_20[1]:
        best_profit_20[0] = ub
        best_profit_20[1] = total_profit_20

print(f"Best Profit (T1 - T0): {best_profit_10}")
print(f"Best Profit (T2 - T1): {best_profit_21}")
print(f"Best Overall Profit (T2 - T0): {best_profit_20}")

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  100
Total Cost at T0: 999999.64
Total Profit from T0 to T1: 181446.66
Total Cost at T1: 1181446.30
Total Profit from T1 to T2: 191201.75
Total Profit from T0 to T2 (Overall): 372648.41

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  200
Total Cost at T0: 999993.45
Total Profit from T0 to T1: 172455.07
Total Cost at T1: 1172448.52
Total Profit from T1 to T2: 249477.79
Total Profit from T0 to T2 (Overall): 421932.86

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  500
Total Cost at T0: 999993.00
Total Profit from T0 to T1: 212224.10
Total Cost at T1: 1212217.10
Total Profit from T1 to T2: 262077.25
Total Profit from T0 to T2 (Overall): 474301.35

Optimal number of shares to buy for each company:
Upper Bound on each allocation (number of shares):  1000
Total Cost at T0: 999989.10
T

In [141]:
df[(df["p20"] >= 0) & (df["p10"] <= 0)]

,Name,join_key,t0,t1,t2,p20,p21,p10,allocation
4,Aaron Industries,-1.0_AARON,263.20,258.78,295.829774,32.629774,37.049774,-4.42,0
14,Alicon Cast.,531147.0_ALICON,1247.65,1243.92,1315.819900,68.169900,71.899900,-3.73,0
15,Amba Enterprises,539196.0___NA__,238.00,235.05,283.694470,45.694470,48.644470,-2.95,0
18,Ami Organics,543349.0_AMIORG,1303.45,1232.86,1328.871825,25.421825,96.011825,-70.59,0
52,Brady & Morris,505690.0___NA__,1250.00,1213.32,1338.553685,88.553685,125.233685,-36.68,0
56,Capacit'e Infra.,540710.0_CAPACITE,310.60,297.35,317.668028,7.068028,20.318028,-13.25,0
59,Cera Sanitary.,532443.0_CERA,8655.35,8537.75,9379.765560,724.415560,842.015560,-117.60,0
62,Colgate-Palmoliv,500830.0_COLPAL,2879.05,2872.05,3289.769952,410.719952,417.719952,-7.00,0
65,Cyber Media Res.,-1.0_CMRSL,172.20,168.65,179.340553,7.140553,10.690553,-3.55,0
67,D-Link India,533146.0_DLINKINDIA,485.10,462.63,540.591442,55.491442,77.961442,-22.47,0


In [142]:
df["allocation"] = allocations[10000]
df.head()

,Name,join_key,t0,t1,t2,p20,p21,p10,allocation
0,20 Microns,533022.0_20MICRONS,225.45,229.42,227.561919,2.111919,-1.858081,3.97,0
1,3M India,523395.0_3MINDIA,37310.00,41655.06,45885.999153,8575.999153,4230.939153,4345.06,0
2,A B B,500002.0_ABB,8531.25,9620.43,10533.824027,2002.574027,913.394027,1089.18,0
3,Aakash Explor.,-1.0_AAKASH,14.16,13.30,-58.956361,-73.116361,-72.256361,-0.86,0
4,Aaron Industries,-1.0_AARON,263.20,258.78,295.829774,32.629774,37.049774,-4.42,0


In [143]:
sum(df["allocation"] > 0)

3

In [144]:
df.to_csv("ATTEMPT 5.csv", index = False)

# Attempt 6
### Reducing risks with a small upper bound on per stock allocation to 1000

In [145]:
df["allocation"] = allocations[1000]
df.head()

,Name,join_key,t0,t1,t2,p20,p21,p10,allocation
0,20 Microns,533022.0_20MICRONS,225.45,229.42,227.561919,2.111919,-1.858081,3.97,0
1,3M India,523395.0_3MINDIA,37310.00,41655.06,45885.999153,8575.999153,4230.939153,4345.06,0
2,A B B,500002.0_ABB,8531.25,9620.43,10533.824027,2002.574027,913.394027,1089.18,0
3,Aakash Explor.,-1.0_AAKASH,14.16,13.30,-58.956361,-73.116361,-72.256361,-0.86,0
4,Aaron Industries,-1.0_AARON,263.20,258.78,295.829774,32.629774,37.049774,-4.42,0


In [146]:
sum(df["allocation"] > 0)

6

In [147]:
df.to_csv("ATTEMPT 6.csv", index = False)